In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader.data import DataReader
import datetime as dt
import pandas_market_calendars as mcal
import plotly.express as px
from sktime.datasets import load_airline
from sktime.forecasting.tbats import TBATS
import requests_cache
from pandas_datareader.yahoo.headers import DEFAULT_HEADERS


from StockPrice.stock_price_visualize import StockData
from StockPrice.stock_price_ml import StockPrediction

ModuleNotFoundError: No module named 'stock_price_visualize'

In [2]:
data = StockData(["Meta", "AMZN", "MSFT", "AAPL", "MCD", "COST", "TM"], "2021-06-01", "2022-06-01")
model = StockPrediction(data)

In [3]:
class StockEvaluate:
    
    def __init__(self, model, asset = 100):
        self.model = model
        self.asset = asset
        self.date = model.date_train
        self.stocks = self.model.stocks
        
        expire_after = dt.timedelta(days=3)
        session = requests_cache.CachedSession(cache_name='cache', expire_after=expire_after)
        session.headers = DEFAULT_HEADERS
    
    def invest(self, date):
        if isinstance(date, str):
            date = pd.Timestamp(date)
        
        ### check last open day is model.date_train
        last_open = StockData.last_open_day(date - pd.Timedelta(days = 1))
        if last_open != self.model.date_train:
            raise ValueError(f"The last day of the train set: {self.model.date_train.strftime('%Y-%m-%d')} "
                             +f"shoud be the last market open day before the given date: {date.strftime('%Y-%m-%d')}")
        
        ### check market open
        if not self.model.check_day_open(date):
            print(f"On {date.strftime('%Y-%m-%d')}, the market is not open.")
            return
        
        ### check whether date already invested
        if self.date >= date:
            print(f"Already invested on the date {date.strftime('%Y-%m-%d')}")
            return
            
        profit_max, stock_best = 0, ""
        self.model.predict()
        for stock in self.stocks:
            open_val = self.get_open(date, stock)
            close_pred = self.model.pred[stock].iloc[-1][stock]
            profit = (close_pred - open_val) / open_val
            
            if profit > profit_max:
                profit_max, stock_best = profit, stock
        
        if profit_max >0:
            new = self.get_return(date, stock_best) * self.asset
            print(f"On {date.strftime('%Y-%m-%d')}, we invest the stock {stock_best}, and now the asset becomes {round(new, 5)}")
            self.asset = new
        else:
            print(f"On {date.strftime('%Y-%m-%d')}, we should not invest, asset keeps {self.asset}")
        
        self.date = date
    
    def get_return(self, date, stock):
        data = DataReader(stock, 'yahoo', date, date)
        val_open, val_close = data.iloc[0]["Open"], data.iloc[0]["Close"]
        
        return val_close / val_open
            
    def get_open(self, date, stock):
        val = DataReader(stock, "yahoo", date, date).iloc[0]["Open"]
        
        return val
    
    def update(self, date):
        self.model.update(date, message = False)
    
    def evaluate(self, days = 10):
        date = self.date
        for _ in range(days):
            date += pd.Timedelta(days = 1)
            self.invest(date)
            self.update(date)
        

In [4]:
eva = StockEvaluate(model, asset = 100)

In [ ]:
eva.evaluate(50)

In [5]:
eva.evaluate(50)

On 2022-06-02, we invest the stock MSFT, and now the asset becomes 103.83058
On 2022-06-03, we invest the stock TM, and now the asset becomes 103.01179
On 2022-06-04, the market is not open.
On 2022-06-05, the market is not open.
On 2022-06-06, we invest the stock AAPL, and now the asset becomes 102.38824
On 2022-06-07, we invest the stock AAPL, and now the asset becomes 105.48081
On 2022-06-08, we invest the stock TM, and now the asset becomes 104.78374
On 2022-06-09, we invest the stock MCD, and now the asset becomes 104.15578
On 2022-06-10, we invest the stock AMZN, and now the asset becomes 100.69372
On 2022-06-11, the market is not open.
On 2022-06-12, the market is not open.
On 2022-06-13, we invest the stock AMZN, and now the asset becomes 100.19116
On 2022-06-14, we invest the stock AMZN, and now the asset becomes 98.38331
On 2022-06-15, we invest the stock AMZN, and now the asset becomes 101.99241
On 2022-06-16, we invest the stock Meta, and now the asset becomes 100.21694
On 

In [80]:
eva.asset

99.90326293253648

In [32]:
eva.invest("2020-10-06")

we invest the stock Meta, and now the asset increased to 107.03717


In [83]:
DataReader("Meta", 'yahoo', "2020-10-01", "2020-10-30")

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-10-01,268.329987,264.799988,265.350006,266.630005,20009800,266.630005
2020-10-02,265.149994,259.179993,261.209991,259.940002,16367600,259.940002
2020-10-05,264.869995,260.839996,262.200012,264.649994,12822300,264.649994
2020-10-06,265.690002,258.239990,261.779999,258.660004,18696900,258.660004
2020-10-07,260.179993,254.820007,259.209991,258.119995,23133400,258.119995
2020-10-08,264.619995,259.149994,259.750000,263.760010,16312800,263.760010
2020-10-09,264.750000,262.170013,264.519989,264.450012,14107800,264.450012
2020-10-12,280.179993,267.869995,270.200012,275.750000,31019300,275.750000
2020-10-13,279.100006,273.390015,277.579987,276.140015,18063300,276.140015


In [52]:
DataReader("Meta", "yahoo", "2022-12-02", "2022-12-02").iloc[0][["Open", "Close"]]

Open     117.830002
Close    123.489998
Name: 2022-12-02 00:00:00, dtype: float64

In [8]:
StockData.last_open_day(pd.Timestamp("2022-12-04"))

Timestamp('2022-12-02 00:00:00')